## dependencies

In [21]:
import matplotlib.pyplot as plt
from skimage import data
from skimage import filters
from skimage import exposure
from skimage import io
from skimage import color
from skimage import morphology
from skimage import measure
from scipy import ndimage

import gc

import time

import numpy as np
import cv2
import ntpath

from os import listdir
import os
from os.path import isfile, isdir, join

## constants & onetimers

In [22]:
EROSION_SIZE = 15
DILATION_SIZE = 15

In [23]:
sift = cv2.xfeatures2d.SIFT_create()
bruteForceMatcher = cv2.BFMatcher()

## Classes

In [24]:
class Frame:
    def __init__(self, filePath):
        
        # load bitmaps
        start = time.time()
        self.filePath = filePath
        self.fileName = ntpath.basename(filePath)
        self.imageSkimage = io.imread(filePath)
        self.grey = color.rgb2grey(self.imageSkimage)
        self.imageCV = cv2.imread(filePath)
        end = time.time()
        self.group = []
        #print (end - start)
        
        # compute masks
        start = time.time()
        finger_mask, screen_mask = self.computeMasks()
        self.maskFinger = finger_mask
        self.maskScreen = screen_mask
        end = time.time()
        #print (end - start)
        
        self.kp = None # compute these when necessary
        self.desc = None # compute these when necessary
        
        self.assignedScene = None
        
        self.thisSceneNumber = None
        
    def dropWeight(self):
        self.imageSkimage = None
        self.grey = None
        self.imageCV = None
        self.maskFinger = None
        self.maskScreen = None
        gc.collect()
    
    def computeMasks(self):
        #locates area usable for image comparison
        
        # Otsu
        start = time.time()
        otsu_tresh = filters.threshold_otsu(self.grey)
        mask = (self.grey > otsu_tresh)
        end = time.time()
        #print (end - start)
        
        
        # Erosion & dilation
        start = time.time()        
        mask = morphology.binary_dilation(mask, morphology.square(DILATION_SIZE)).astype(int)
        mask = morphology.binary_erosion(mask, morphology.square(EROSION_SIZE))
        end = time.time()
        #print (end - start)        
        
        #For testing
        #mask = np.array([[1, 1, 1, 0],
        #                [1, 0, 0, 0],
        #                 [0, 0, 1, 0],
        #                 [0, 0, 1, 0],
        #                 [0, 0, 0, 0]])
        
        # Select largest component (screen)
        start = time.time()
        mask = measure.label(mask)
        regions = measure.regionprops(mask)
        largest_region_props = max(regions, key=lambda r: r.area)
        mask = (mask == largest_region_props.label).astype(int)
        mask = ndimage.binary_fill_holes(mask).astype(int)
        end = time.time()
        #print (end - start)

        # Get final masks (ignore fingers as much as possible)
        start = time.time() 
        finger_mask = mask
        #screen_mask = (morphology.convex_hull_image(mask)).astype(int) #takes too much time
        screen_mask = None
        end = time.time()
        #print (end - start)
        
        return finger_mask, screen_mask
    
    def computeSIFTDescriptors(self):
        start = time.time()
        gray = cv2.cvtColor(self.imageCV, cv2.COLOR_BGR2GRAY)
        self.kp, self.desc = sift.detectAndCompute(gray, mask=self.maskFinger.astype(np.uint8))
        end = time.time()
        #print ('SIFT descriptors ' + str(end - start))        
        return self.kp, self.desc
    
    def displayMasks(self):
        plt.figure(figsize=(9, 4))
        plt.subplot(131)
        plt.imshow(self.imageSkimage, cmap='gray', interpolation='nearest')
        plt.axis('off')
        plt.subplot(132)
        plt.imshow(self.maskFinger, cmap='gray', interpolation='nearest')
        plt.axis('off')
        plt.subplot(133)
        plt.imshow(self.maskScreen, cmap='gray', interpolation='nearest')
        plt.axis('off')

        plt.tight_layout()
        plt.show()

        
class EmptyFrame(Frame):
    def __init__(self):
        self.fileName = 'none'
        self.thisSceneNumber = 0
        
emptyFrame = EmptyFrame()

## routines

In [25]:
def similaritySIFT(frame, scene):
    start = time.time()
    matches = bruteForceMatcher.knnMatch(frame.desc, scene.desc, k=2)
    total = 0
    good = 0
    for m,n in matches:
        total +=1
        if m.distance < 0.75*n.distance:
            good += 1
    end = time.time()
    #print ('SIFT similarity duration ' + str(end - start))
    return good/total

In [26]:
def similaritySSIM(frame, scene):
    start = time.time()
    maskedFrame = frame.grey * frame.maskFinger
    maskedScene = scene.grey * frame.maskFinger #yes, frame's mask must be used
    res = measure.compare_ssim(maskedFrame, maskedScene, None)
    end = time.time()
    #print ('SSIM similarity duration ' + str(end - start))
    return res

# Main experiment

## Scene occurence computation scrip:

In [28]:
# SELECT EXPERIMENT SAMPLE
#sample_path = 'experiment-samples/extra-small'
#sample_path = 'experiment-samples/trivial'
sample_path = 'experiment-samples-nogit/medium'


# LOAD AND PREPROCESS SCENES (defined by the experiment analyst)

scenes_path = sample_path + '/scenes'
scene_files = [(scenes_path+'/'+f) for f in listdir(scenes_path) if isfile(join(scenes_path, f))]
scenes = []
for path in scene_files:
    sceneFrame = Frame(path)
    sceneFrame.computeSIFTDescriptors()
    sceneFrame.thisSceneNumber = int(os.path.splitext(sceneFrame.fileName)[0])
    print('loaded scene '+str(sceneFrame.thisSceneNumber))
    scenes.append(sceneFrame)

# PROCESS EACH RECORDING
recordings_path = sample_path + '/recordings'
recordings = [d for d in listdir(recordings_path) if isdir(join(recordings_path, d))]
recordingsResults = []
for recording in recordings:
    recording_path = recordings_path + '/' + recording
    frame_files = [(recording_path+'/'+f) for f in listdir(recording_path) if isfile(join(recording_path, f))]
    frames = []
    previousFrame = None
    recordingsResults.append(frames)
    # PROCESS EACH FRAME
    for path in frame_files:
        #PREPROCESS FRAME
        frame = Frame(path) # Preprocessing takes place here
        print('loaded frame ' + frame.fileName)
        frames.append(frame)
        
        #FRAME GROUPING (not implemented in this implementation yet)
        if (previousFrame != None):
            simSSIM = similaritySSIM(frame, previousFrame)
            if (simSSIM > 0.98):
                frame.dropWeight() # drop bitmaps, memory preservation
                frame.assignedScene = previousFrame.assignedScene
                previousFrame.group.append(frame)
                #purposefully, we keep the previous frame at the head of the group
                continue
        
        # COMPUTE SIMILARITY AND ASSIGN SCENE
        bestSum = 0
        bestScene = emptyFrame
        frame.computeSIFTDescriptors()
        for scene in scenes:
            simSIFT = similaritySIFT(frame, scene)
            simSSIM = similaritySSIM(frame, scene)
            if(bestSum < simSIFT + simSSIM and (1.15 < simSIFT + simSSIM or 0.45 < simSIFT)):
                bestSum = simSIFT + simSSIM
                bestScene = scene 
            #print ("SIFT: " + str(simSIFT))
            #print ("SSIM: " + str(simSSIM))
        frame.assignedScene = bestScene
                
        # Tidy up (preventing memory leaks) by removing unnecessary blobs
        if(previousFrame != None):
            previousFrame.dropWeight()
            
        
        # very last thing before we proceed to next frame
        previousFrame = frame
        #print ('     '+frame.assignedScene.fileName)

        
    # FILTER ORPHANS 
    for frameNumber, currentFrame in enumerate(frames):
        if frameNumber == 0 : continue
        if frameNumber == len(frames)-1: continue
        previousScene = frames[frameNumber-1].assignedScene
        currentScene  = frames[frameNumber  ].assignedScene # (currentFrame)
        nextScene     = frames[frameNumber+1].assignedScene
        # Rule 1:
        if (currentScene is not previousScene) and (previousScene is nextScene):
            currentFrame.assignedScene = previousScene
            print('Rule 1 applied on frame '+str(frameNumber))
            continue
        # Rule 2:
        if (currentScene is not previousScene) and (currentScene is not nextScene) and (currentScene is not emptyFrame):
            currentFrame.assignedScene = emptyFrame
            print('Rule 2 applied on frame '+str(frameNumber))
            continue
    


loaded scene 1
loaded scene 10
loaded scene 11
loaded scene 12
loaded scene 2
loaded scene 3
loaded scene 4
loaded scene 5
loaded scene 6
loaded scene 7
loaded scene 8
loaded scene 9
loaded frame pabk_01_01000.png
loaded frame pabk_01_01001.png
loaded frame pabk_01_01002.png
loaded frame pabk_01_01003.png
loaded frame pabk_01_01004.png
loaded frame pabk_01_01005.png
loaded frame pabk_01_01006.png
loaded frame pabk_01_01007.png
loaded frame pabk_01_01008.png
loaded frame pabk_01_01009.png
loaded frame pabk_01_01010.png
loaded frame pabk_01_01011.png
loaded frame pabk_01_01012.png
loaded frame pabk_01_01013.png
loaded frame pabk_01_01014.png
loaded frame pabk_01_01015.png
loaded frame pabk_01_01016.png
loaded frame pabk_01_01017.png
loaded frame pabk_01_01018.png
loaded frame pabk_01_01019.png
loaded frame pabk_01_01020.png
loaded frame pabk_01_01021.png
loaded frame pabk_01_01022.png
loaded frame pabk_01_01023.png
loaded frame pabk_01_01024.png
loaded frame pabk_01_01025.png
loaded fram

loaded frame pabk_01_01259.png
loaded frame pabk_01_01260.png
loaded frame pabk_01_01261.png
loaded frame pabk_01_01262.png
loaded frame pabk_01_01263.png
loaded frame pabk_01_01264.png
loaded frame pabk_01_01265.png
loaded frame pabk_01_01266.png
loaded frame pabk_01_01267.png
loaded frame pabk_01_01268.png
loaded frame pabk_01_01269.png
loaded frame pabk_01_01270.png
loaded frame pabk_01_01271.png
loaded frame pabk_01_01272.png
loaded frame pabk_01_01273.png
loaded frame pabk_01_01274.png
loaded frame pabk_01_01275.png
loaded frame pabk_01_01276.png
loaded frame pabk_01_01277.png
loaded frame pabk_01_01278.png
loaded frame pabk_01_01279.png
loaded frame pabk_01_01280.png
loaded frame pabk_01_01281.png
loaded frame pabk_01_01282.png
loaded frame pabk_01_01283.png
loaded frame pabk_01_01284.png
loaded frame pabk_01_01285.png
loaded frame pabk_01_01286.png
loaded frame pabk_01_01287.png
loaded frame pabk_01_01288.png
loaded frame pabk_01_01289.png
loaded frame pabk_01_01290.png
loaded f

loaded frame pabk_01_01524.png
loaded frame pabk_01_01525.png
loaded frame pabk_01_01526.png
loaded frame pabk_01_01527.png
loaded frame pabk_01_01528.png
loaded frame pabk_01_01529.png
loaded frame pabk_01_01530.png
loaded frame pabk_01_01531.png
loaded frame pabk_01_01532.png
loaded frame pabk_01_01533.png
loaded frame pabk_01_01534.png
loaded frame pabk_01_01535.png
loaded frame pabk_01_01536.png
loaded frame pabk_01_01537.png
loaded frame pabk_01_01538.png
loaded frame pabk_01_01539.png
loaded frame pabk_01_01540.png
loaded frame pabk_01_01541.png
loaded frame pabk_01_01542.png
loaded frame pabk_01_01543.png
loaded frame pabk_01_01544.png
loaded frame pabk_01_01545.png
loaded frame pabk_01_01546.png
loaded frame pabk_01_01547.png
loaded frame pabk_01_01548.png
loaded frame pabk_01_01549.png
loaded frame pabk_01_01550.png
loaded frame pabk_01_01551.png
loaded frame pabk_01_01552.png
loaded frame pabk_01_01553.png
loaded frame pabk_01_01554.png
loaded frame pabk_01_01555.png
loaded f

loaded frame pabk_01_01789.png
loaded frame pabk_01_01790.png
loaded frame pabk_01_01791.png
loaded frame pabk_01_01792.png
loaded frame pabk_01_01793.png
loaded frame pabk_01_01794.png
loaded frame pabk_01_01795.png
loaded frame pabk_01_01796.png
loaded frame pabk_01_01797.png
loaded frame pabk_01_01798.png
loaded frame pabk_01_01799.png
loaded frame pabk_01_01800.png
loaded frame pabk_01_01801.png
loaded frame pabk_01_01802.png
loaded frame pabk_01_01803.png
loaded frame pabk_01_01804.png
loaded frame pabk_01_01805.png
loaded frame pabk_01_01806.png
loaded frame pabk_01_01807.png
loaded frame pabk_01_01808.png
loaded frame pabk_01_01809.png
loaded frame pabk_01_01810.png
loaded frame pabk_01_01811.png
loaded frame pabk_01_01812.png
loaded frame pabk_01_01813.png
loaded frame pabk_01_01814.png
loaded frame pabk_01_01815.png
loaded frame pabk_01_01816.png
loaded frame pabk_01_01817.png
loaded frame pabk_01_01818.png
loaded frame pabk_01_01819.png
loaded frame pabk_01_01820.png
loaded f

## Evaluation sequence

In [31]:
import pandas as pd
labels = pd.read_excel(sample_path+'/labels.xlsx', sheetname='labels')
labels['scene'] = pd.to_numeric(labels['scene'].replace('none','0'))
labels = labels.set_index('frame')
labels = (labels.to_dict())['scene']

totalCount = 0
totalCorrect = 0
for frames in recordingsResults:
    for frame in frames:
        totalCount += 1
        #print(type(frame.assignedScene.thisSceneNumber).__name__ + ' ' + type(labels[frame.fileName]).__name__)
        if(frame.assignedScene == None): continue
        if(frame.assignedScene.thisSceneNumber == labels[frame.fileName]):
            totalCorrect += 1

totalAccuracy = totalCorrect / totalCount

print('total:    ' + str(totalCount))
print('correct:  ' + str(totalCorrect))
print('accuracy: ' + str(totalAccuracy))


total:    1000
correct:  971
accuracy: 0.971


In [32]:
for frames in recordingsResults:
    for frame in frames:
        a = frame.fileName
        b = str(frame.assignedScene.thisSceneNumber)
        c = str(labels[frame.fileName])
        print(a +' '+ b + ' ' + c)

pabk_01_01000.png 3 3
pabk_01_01001.png 3 3
pabk_01_01002.png 3 3
pabk_01_01003.png 3 3
pabk_01_01004.png 3 3
pabk_01_01005.png 3 3
pabk_01_01006.png 3 3
pabk_01_01007.png 3 3
pabk_01_01008.png 3 3
pabk_01_01009.png 3 3
pabk_01_01010.png 3 3
pabk_01_01011.png 3 3
pabk_01_01012.png 3 3
pabk_01_01013.png 3 3
pabk_01_01014.png 3 3
pabk_01_01015.png 3 3
pabk_01_01016.png 3 3
pabk_01_01017.png 3 3
pabk_01_01018.png 3 3
pabk_01_01019.png 3 3
pabk_01_01020.png 3 3
pabk_01_01021.png 3 3
pabk_01_01022.png 3 3
pabk_01_01023.png 3 3
pabk_01_01024.png 3 3
pabk_01_01025.png 3 3
pabk_01_01026.png 3 3
pabk_01_01027.png 3 3
pabk_01_01028.png 3 3
pabk_01_01029.png 3 3
pabk_01_01030.png 3 3
pabk_01_01031.png 3 3
pabk_01_01032.png 3 3
pabk_01_01033.png 3 3
pabk_01_01034.png 3 3
pabk_01_01035.png 3 3
pabk_01_01036.png 3 3
pabk_01_01037.png 3 3
pabk_01_01038.png 3 3
pabk_01_01039.png 3 3
pabk_01_01040.png 3 3
pabk_01_01041.png 3 3
pabk_01_01042.png 3 3
pabk_01_01043.png 3 3
pabk_01_01044.png 3 3
pabk_01_01